In [29]:
#install pyreadstat ==> to read spss file
!pip install pyreadstat

Defaulting to user installation because normal site-packages is not writeable


In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import pyreadstat as pyrs
library(haven)
#set no limits on data display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('use_inf_as_na', True)
#getting the work directory
import os 
cwd = os.getcwd()
print(cwd)

C:\Users\OMEN\Anaconda3 Files


C:\Users\OMEN\AppData\Local\Temp\ipykernel_1364\3450391093.py:9: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


In [2]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [24]:
!pip install haven

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.6 MB 5.9 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.6 MB 6.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.6 MB 8.0 MB/s eta 0:00:01
   ----------------------------------- ---- 1.4/1.6 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/181.4 kB ? eta -:--:--
   --------------------------------------- 181.4/181.4 kB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/297.0 kB ? eta -:--:--
   --------------------------------------  297.0/297.0 kB 17.9 MB/s eta 0:00:01
   ---------------------------------------- 297.0/297.0 kB 9.0 MB/s eta 0:00:00


In [2]:
#importing raw_per table from SAV to dataframe.
df_16_raw_per, meta_16_raw_per = pyrs.read_sav("UK LCF Data Sets/2016_17_rawper_ukanon.sav")
df_17_raw_per, meta_17_raw_per = pyrs.read_sav("UK LCF Data Sets/2017_rawper_ukanon_2017-18.sav")
df_18_raw_per, meta_18_raw_per = pyrs.read_sav("UK LCF Data Sets/2018_rawper_ukanon_final.sav")
df_19_raw_per, meta_19_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2019_rawper_ukanon_final.sav")
df_20_raw_per, meta_20_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2020_rawper_ukanon_final.sav")
df_21_raw_per, meta_21_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2021_rawper_ukanon_final.sav")
df_22_raw_per, meta_22_raw_per = pyrs.read_sav("UK LCF Data Sets/lcfs_2022_rawper_ukanon_final.sav")

#rename column WfhCor2 to WfhCor to match other tables
df_21_raw_per.rename(columns={"WfhCor2": "WfhCor"}, inplace=True)
df_22_raw_per.rename(columns={"wfhcor2": "WfhCor"}, inplace=True)

In [3]:
#Create WfhCor columns for dataset prior 2020, set to 0
df_16_raw_per['WfhCor'] = 0
df_17_raw_per['WfhCor'] = 0 
df_18_raw_per['WfhCor'] = 0 
df_19_raw_per['WfhCor'] = 0 

In [4]:
#setting NaN values to 2 (which means no WFH).
#w['female'] = w['female'].map({'female': 1, 'male': 0})
#df_20_raw_per['WfhCor'] = df_20_raw_per['WfhCor'].fillna(0)
df_20_raw_per['WfhCor'] = df_20_raw_per['WfhCor'].fillna(2)
df_20_raw_per['WfhCor'] = df_20_raw_per['WfhCor'].map({1:1, 2:0})
df_21_raw_per['WfhCor'] = df_21_raw_per['WfhCor'].fillna(2)
df_21_raw_per['WfhCor'] = df_21_raw_per['WfhCor'].map({1:1, 2:0})
df_22_raw_per['WfhCor'] = df_22_raw_per['WfhCor'].fillna(2)
df_22_raw_per['WfhCor'] = df_22_raw_per['WfhCor'].map({1:1, 2:0})

In [5]:
# Drop ClaWfhCor_p

#df_20_raw_per = df_20_raw_per.drop('ClaWfhCor_p', axis=1)
df_21_raw_per = df_21_raw_per.drop('ClaWfhCor_p', axis=1)
df_22_raw_per = df_22_raw_per.drop('ClaWfhCor_p', axis=1)

In [6]:
print('2020 \n')
print(df_20_raw_per['WfhCor'].value_counts())
print('\n 2021 \n')
print(df_21_raw_per['WfhCor'].value_counts())
print('\n 2022 \n')
print(df_22_raw_per['WfhCor'].value_counts())

2020 

WfhCor
0    9837
1    2400
Name: count, dtype: int64

 2021 

WfhCor
0    10466
1     2376
Name: count, dtype: int64

 2022 

WfhCor
0    8908
1    1167
Name: count, dtype: int64


new code v9

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

class WorkFromHomeFeatureSelector:
    def __init__(self, data, target_column):
        """
        Initialize the feature selector for work-from-home analysis
        
        Args:
            data: pandas DataFrame with survey data
            target_column: string name of the target variable (work from home)
        """
        self.data = data.copy()
        self.target_column = target_column
        self.X = None
        self.y = None
        self.feature_scores = {}
        self.selected_features = {}
        
    def preprocess_data(self):
        """Preprocess the survey data for machine learning"""
        print("Preprocessing data...")
        
        # Separate features and target
        self.y = self.data[self.target_column]
        self.X = self.data.drop(columns=[self.target_column])
        
        # Handle categorical variables
        categorical_cols = self.X.select_dtypes(include=['object']).columns
        label_encoders = {}
        
        for col in categorical_cols:
            le = LabelEncoder()
            self.X[col] = le.fit_transform(self.X[col].astype(str))
            label_encoders[col] = le
        
        # Handle missing values
        self.X = self.X.fillna(self.X.median())
        
        # Encode target variable if it's categorical
        if self.y.dtype == 'object':
            le_target = LabelEncoder()
            self.y = le_target.fit_transform(self.y)
        
        print(f"Data shape: {self.X.shape}")
        print(f"Target distribution:\n{pd.Series(self.y).value_counts()}")
        
        return label_encoders
    
    def mutual_information_selection(self, k=20):
        """Select features based on mutual information"""
        print(f"\n=== Mutual Information Feature Selection (top {k}) ===")
        
        mi_scores = mutual_info_classif(self.X, self.y, random_state=42)
        mi_scores_df = pd.DataFrame({
            'feature': self.X.columns,
            'mi_score': mi_scores
        }).sort_values('mi_score', ascending=False)
        
        top_features = mi_scores_df.head(k)['feature'].tolist()
        self.selected_features['mutual_info'] = top_features
        self.feature_scores['mutual_info'] = mi_scores_df
        
        print("Top 10 features by Mutual Information:")
        print(mi_scores_df.head(10))
        
        return top_features
    
    def random_forest_selection(self, n_estimators=100, max_features=20):
        """Select features using Random Forest importance"""
        print(f"\n=== Random Forest Feature Selection (top {max_features}) ===")
        
        rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42, n_jobs=-1)
        rf.fit(self.X, self.y)
        
        importance_df = pd.DataFrame({
            'feature': self.X.columns,
            'importance': rf.feature_importances_
        }).sort_values('importance', ascending=False)
        
        top_features = importance_df.head(max_features)['feature'].tolist()
        self.selected_features['random_forest'] = top_features
        self.feature_scores['random_forest'] = importance_df
        
        print("Top 10 features by Random Forest importance:")
        print(importance_df.head(10))
        
        return top_features
    
    def xgboost_selection(self, max_features=20):
        """Feature selection using XGBoost importance"""
        print(f"\n=== XGBoost Feature Selection (top {max_features}) ===")
        
        xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
        xgb_model.fit(self.X, self.y)
        
        importance_df = pd.DataFrame({
            'feature': self.X.columns,
            'importance': xgb_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        top_features = importance_df.head(max_features)['feature'].tolist()
        self.selected_features['xgboost'] = top_features
        self.feature_scores['xgboost'] = importance_df
        
        print("Top 10 features by XGBoost importance:")
        print(importance_df.head(10))
        
        return top_features
    
    def get_consensus_features(self, min_methods=2):
        """Get features selected by multiple methods"""
        print(f"\n=== Consensus Features (selected by at least {min_methods} methods) ===")
        
        all_features = []
        for method, features in self.selected_features.items():
            all_features.extend(features)
        
        feature_counts = pd.Series(all_features).value_counts()
        consensus_features = feature_counts[feature_counts >= min_methods].index.tolist()
        
        print(f"Features selected by {min_methods}+ methods:")
        for feature in consensus_features:
            count = feature_counts[feature]
            print(f"  {feature}: selected by {count} methods")
        
        self.selected_features['consensus'] = consensus_features
        return consensus_features
    
    def evaluate_feature_sets(self):
        """Evaluate different feature selection methods"""
        print(f"\n=== Evaluating Feature Selection Methods ===")
        
        results = {}
        
        for method, features in self.selected_features.items():
            if len(features) == 0:
                continue
                
            # Select features
            X_selected = self.X[features]
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X_selected, self.y, test_size=0.2, random_state=42, stratify=self.y
            )
            
            # Train Random Forest
            rf = RandomForestClassifier(n_estimators=100, random_state=42)
            rf.fit(X_train, y_train)
            
            # Evaluate
            y_pred = rf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            cv_scores = cross_val_score(rf, X_selected, self.y, cv=5)
            
            results[method] = {
                'n_features': len(features),
                'test_accuracy': accuracy,
                'cv_mean': cv_scores.mean(),
                'cv_std': cv_scores.std()
            }
            
            print(f"{method}: {len(features)} features, "
                  f"CV Score: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}, "
                  f"Test Accuracy: {accuracy:.3f}")
        
        return results
    
    def plot_feature_importance(self, method='random_forest', top_n=15):
        """Plot feature importance for a given method"""
        if method not in self.feature_scores:
            print(f"Method {method} not found. Available methods: {list(self.feature_scores.keys())}")
            return
        
        plt.figure(figsize=(10, 8))
        
        if method == 'random_forest' or method == 'xgboost':
            data = self.feature_scores[method].head(top_n)
            plt.barh(data['feature'], data['importance'])
            plt.xlabel('Feature Importance')
        elif method == 'mutual_info':
            data = self.feature_scores[method].head(top_n)
            plt.barh(data['feature'], data['mi_score'])
            plt.xlabel('Mutual Information Score')
        
        plt.title(f'Top {top_n} Features - {method.title()}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
    
    def train_final_model(self, method='consensus', model_type='random_forest'):
        """Train the final model using selected features"""
        print(f"\n=== Training Final Model ===")
        
        # Get selected features
        if method not in self.selected_features:
            print(f"Method {method} not found. Using random_forest method.")
            method = 'random_forest'
        
        selected_features = self.selected_features[method]
        print(f"Training with {len(selected_features)} features from {method} method")
        
        # Prepare data
        X_selected = self.X[selected_features]
        
        # Split for final evaluation
        X_train, X_test, y_train, y_test = train_test_split(
            X_selected, self.y, test_size=0.2, random_state=42, stratify=self.y
        )
        
        # Choose model
        if model_type == 'random_forest':
            model = RandomForestClassifier(
                n_estimators=200, 
                max_depth=10, 
                min_samples_split=5,
                random_state=42, 
                n_jobs=-1
            )
        elif model_type == 'xgboost':
            model = xgb.XGBClassifier(
                n_estimators=200,
                max_depth=6,
                learning_rate=0.1,
                random_state=42,
                eval_metric='logloss'
            )
        elif model_type == 'logistic':
            # Scale features for logistic regression
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            model = LogisticRegression(random_state=42, max_iter=1000)
            model.fit(X_train_scaled, y_train)
            
            # Store scaler for prediction
            self.scaler = scaler
            self.final_model = model
            self.final_features = selected_features
            self.model_type = model_type
            
            # Evaluate
            y_pred = model.predict(X_test_scaled)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Final model test accuracy: {accuracy:.3f}")
            print("\nClassification Report:")
            print(classification_report(y_test, y_pred))
            
            return model, selected_features
        
        # Train model (for non-logistic models)
        model.fit(X_train, y_train)
        
        # Store model and features for prediction
        self.final_model = model
        self.final_features = selected_features
        self.model_type = model_type
        if model_type == 'logistic':
            # Scaler already stored above
            pass
        else:
            self.scaler = None
        
        # Evaluate
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"Final model test accuracy: {accuracy:.3f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        return model, selected_features
    
    def save_model_pipeline(self, filepath):
        """Save the complete model pipeline for future use"""
        import pickle
        
        pipeline = {
            'model': self.final_model,
            'features': self.final_features,
            'model_type': self.model_type,
            'scaler': self.scaler,
            'label_encoders': self.label_encoders,
            'feature_medians': self.X[self.final_features].median().to_dict()
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(pipeline, f)
        
        print(f"Model pipeline saved to {filepath}")
    
    def predict_new_data(self, new_data, handle_missing_features='impute'):
        """
        Predict work-from-home capability for new data
        
        Args:
            new_data: pandas DataFrame (may be missing some features)
            handle_missing_features: str, how to handle missing features
                - 'impute': Fill missing features with default values
                - 'retrain': Use subset of available features and retrain
                - 'error': Raise error if features are missing
        
        Returns:
            predictions: array of predictions (0/1)
            probabilities: array of prediction probabilities
            feature_status: dict with information about missing features
        """
        if not hasattr(self, 'final_model'):
            raise ValueError("No trained model found. Run train_final_model() first.")
        
        print(f"Making predictions for {len(new_data)} samples...")
        
        # Check for missing features
        missing_features = set(self.final_features) - set(new_data.columns)
        available_features = set(self.final_features) & set(new_data.columns)
        
        feature_status = {
            'required_features': self.final_features,
            'available_features': list(available_features),
            'missing_features': list(missing_features),
            'missing_count': len(missing_features),
            'coverage': len(available_features) / len(self.final_features)
        }
        
        print(f"Feature coverage: {feature_status['coverage']:.1%} ({len(available_features)}/{len(self.final_features)})")
        
        if missing_features:
            print(f"Missing features: {list(missing_features)}")
            
            if handle_missing_features == 'error':
                raise ValueError(f"Missing required features: {list(missing_features)}")
            
            elif handle_missing_features == 'retrain':
                return self._predict_with_subset_features(new_data, list(available_features), feature_status)
            
            elif handle_missing_features == 'impute':
                new_data_processed = self._preprocess_new_data_with_imputation(new_data, missing_features)
            else:
                raise ValueError("handle_missing_features must be 'impute', 'retrain', or 'error'")
        else:
            # All features available
            new_data_processed = self._preprocess_new_data(new_data)
        
        # Select only the features used in training
        X_new = new_data_processed[self.final_features]

        # Handle NaN values before scaling/prediction
        if X_new.isnull().any().any():
            print("Warning: Input contains NaN values. Handling them...")
            
            # Show rows with NaN values
            nan_rows = X_new.isnull().any(axis=1)

                
            # Option 1: Drop rows with NaN (if you can afford to lose some data)
            #X_new = X_new.dropna()
            
            # Option 2: Fill NaN with appropriate values
            # For numerical columns - use mean, median, or 0
            X_new_clean = X_new.fillna(X_new.mean())  # or .median() or 0
            X_new = X_new_clean.copy()
            # Option 3: Use the same imputation strategy used during training
            # (You should have saved the imputer used during training)
            # X_new_clean = self.imputer.transform(X_new)
            
        else:
            X_new
    
        # Scale if needed
        if self.scaler is not None:
            X_new_scaled = self.scaler.transform(X_new)
            predictions = self.final_model.predict(X_new_scaled)
            probabilities = self.final_model.predict_proba(X_new_scaled)[:, 1]
        else:
            predictions = self.final_model.predict(X_new)
            probabilities = self.final_model.predict_proba(X_new)[:, 1]
        
        return predictions, probabilities, feature_status
    
    def _preprocess_new_data_with_imputation(self, new_data, missing_features):
        """Preprocess new data and impute missing features"""
        new_data_copy = self._preprocess_new_data(new_data)
        
        print(f"Imputing {len(missing_features)} missing features...")
        
        # Impute missing features using various strategies
        for feature in missing_features:
            if feature in self.X.columns:
                # Use training data statistics for imputation
                if self.X[feature].dtype in ['int64', 'float64']:
                    # Numerical feature - use median
                    imputed_value = self.X[feature].median()
                    print(f"  {feature}: imputed with median value {imputed_value}")
                else:
                    # Categorical feature - use mode
                    imputed_value = self.X[feature].mode().iloc[0]
                    print(f"  {feature}: imputed with mode value '{imputed_value}'")
                
                new_data_copy[feature] = imputed_value
            else:
                # Feature not in training data - use neutral value
                print(f"  {feature}: unknown feature, imputing with 0")
                new_data_copy[feature] = 0
        
        return new_data_copy
    
    def _predict_with_subset_features(self, new_data, available_features, feature_status):
        """Retrain model with available features and make predictions"""
        print(f"Retraining model with {len(available_features)} available features...")
        
        if len(available_features) < 3:
            raise ValueError(f"Too few features available ({len(available_features)}). Need at least 3 features for reliable predictions.")
        
        # Preprocess new data
        new_data_processed = self._preprocess_new_data(new_data)
        
        # Train a new model with available features only
        X_subset = self.X[available_features]
        
        # Use same model type as original
        if self.model_type == 'random_forest':
            subset_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        elif self.model_type == 'xgboost':
            subset_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
        elif self.model_type == 'logistic':
            subset_model = LogisticRegression(random_state=42, max_iter=1000)
            # Scale features for logistic regression
            scaler = StandardScaler()
            X_subset_scaled = scaler.fit_transform(X_subset)
            subset_model.fit(X_subset_scaled, self.y)
            
            # Make predictions
            X_new_subset = new_data_processed[available_features]
            X_new_scaled = scaler.transform(X_new_subset)
            predictions = subset_model.predict(X_new_scaled)
            probabilities = subset_model.predict_proba(X_new_scaled)[:, 1]
            
            # Add warning about reduced performance
            feature_status['warning'] = f"Model retrained with {len(available_features)}/{len(self.final_features)} features. Performance may be reduced."
            
            return predictions, probabilities, feature_status
        
        # For non-logistic models
        subset_model.fit(X_subset, self.y)
        
        # Make predictions
        X_new_subset = new_data_processed[available_features]
        predictions = subset_model.predict(X_new_subset)
        probabilities = subset_model.predict_proba(X_new_subset)[:, 1]
        
        # Add warning about reduced performance
        feature_status['warning'] = f"Model retrained with {len(available_features)}/{len(self.final_features)} features. Performance may be reduced."
        
        return predictions, probabilities, feature_status
    
    def _preprocess_new_data(self, new_data):
        """Preprocess new data using the same transformations as training data"""
        new_data_copy = new_data.copy()
        
        # Apply label encoders to categorical variables
        if hasattr(self, 'label_encoders'):
            for col, encoder in self.label_encoders.items():
                if col in new_data_copy.columns:
                    # Handle unseen categories
                    try:
                        new_data_copy[col] = encoder.transform(new_data_copy[col].astype(str))
                    except ValueError:
                        # For unseen categories, assign the most frequent class
                        print(f"Warning: Unseen categories in {col}. Using most frequent class.")
                        most_frequent = encoder.classes_[0]  # First class (most frequent during fit)
                        new_data_copy[col] = new_data_copy[col].apply(
                            lambda x: x if str(x) in encoder.classes_ else most_frequent
                        )
                        new_data_copy[col] = encoder.transform(new_data_copy[col].astype(str))
        
        # Handle missing values using training medians
        if hasattr(self, 'feature_medians'):
            for col in new_data_copy.columns:
                if col in self.feature_medians:
                    new_data_copy[col] = new_data_copy[col].fillna(self.feature_medians[col])
                else:
                    new_data_copy[col] = new_data_copy[col].fillna(new_data_copy[col].median())
        
        return new_data_copy
    
    def get_feature_importance_for_missing(self, missing_features):
        """Get importance scores for missing features to assess impact"""
        if not hasattr(self, 'feature_scores'):
            return None
        
        importance_info = {}
        
        # Check Random Forest importance
        if 'random_forest' in self.feature_scores:
            rf_scores = self.feature_scores['random_forest'].set_index('feature')
            for feature in missing_features:
                if feature in rf_scores.index:
                    importance_info[feature] = {
                        'rf_importance': rf_scores.loc[feature, 'importance'],
                        'rf_rank': rf_scores.index.get_loc(feature) + 1
                    }
        
        # Check XGBoost importance
        if 'xgboost' in self.feature_scores:
            xgb_scores = self.feature_scores['xgboost'].set_index('feature')
            for feature in missing_features:
                if feature in xgb_scores.index:
                    if feature not in importance_info:
                        importance_info[feature] = {}
                    importance_info[feature]['xgb_importance'] = xgb_scores.loc[feature, 'importance']
                    importance_info[feature]['xgb_rank'] = xgb_scores.index.get_loc(feature) + 1
        
        # Check Mutual Information
        if 'mutual_info' in self.feature_scores:
            mi_scores = self.feature_scores['mutual_info'].set_index('feature')
            for feature in missing_features:
                if feature in mi_scores.index:
                    if feature not in importance_info:
                        importance_info[feature] = {}
                    importance_info[feature]['mi_score'] = mi_scores.loc[feature, 'mi_score']
                    importance_info[feature]['mi_rank'] = mi_scores.index.get_loc(feature) + 1
        
        return importance_info
    
    def run_complete_analysis(self):
        """Run the complete feature selection pipeline"""
        print("="*60)
        print("WORK FROM HOME FEATURE SELECTION ANALYSIS")
        print("="*60)
        
        # Preprocess data
        self.label_encoders = self.preprocess_data()
        
        # Run feature selection methods
        self.mutual_information_selection()
        self.random_forest_selection()
        self.xgboost_selection()
        
        # Get consensus features
        self.get_consensus_features()
        
        # Evaluate methods
        results = self.evaluate_feature_sets()
        
        return results

def load_saved_model(filepath):
    """Load a previously saved model pipeline"""
    import pickle
    
    with open(filepath, 'rb') as f:
        pipeline = pickle.load(f)
    
    return pipeline

def predict_with_saved_model(pipeline, new_data, handle_missing_features='impute'):
    """Make predictions using a saved model pipeline with missing feature handling"""
    model = pipeline['model']
    features = pipeline['features']
    scaler = pipeline['scaler']
    label_encoders = pipeline['label_encoders']
    feature_medians = pipeline['feature_medians']
    
    # Check for missing features
    missing_features = set(features) - set(new_data.columns)
    available_features = set(features) & set(new_data.columns)
    
    feature_status = {
        'required_features': features,
        'available_features': list(available_features),
        'missing_features': list(missing_features),
        'missing_count': len(missing_features),
        'coverage': len(available_features) / len(features)
    }
    
    print(f"Feature coverage: {feature_status['coverage']:.1%} ({len(available_features)}/{len(features)})")
    
    if missing_features:
        print(f"Missing features: {list(missing_features)}")
        
        if handle_missing_features == 'error':
            raise ValueError(f"Missing required features: {list(missing_features)}")
        elif handle_missing_features == 'impute':
            new_data_copy = _impute_missing_features_saved_model(
                new_data, missing_features, label_encoders, feature_medians, features
            )
        else:
            raise ValueError("For saved models, only 'impute' and 'error' modes are supported")
    else:
        new_data_copy = new_data.copy()
    
    # Preprocess data
    for col, encoder in label_encoders.items():
        if col in new_data_copy.columns:
            try:
                new_data_copy[col] = encoder.transform(new_data_copy[col].astype(str))
            except ValueError:
                most_frequent = encoder.classes_[0]
                new_data_copy[col] = new_data_copy[col].apply(
                    lambda x: x if str(x) in encoder.classes_ else most_frequent
                )
                new_data_copy[col] = encoder.transform(new_data_copy[col].astype(str))
    
    # Handle missing values
    for col in features:
        if col in new_data_copy.columns:
            new_data_copy[col] = new_data_copy[col].fillna(feature_medians.get(col, 0))
    
    # Select features and make predictions
    X_new = new_data_copy[features]
    
    if scaler is not None:
        X_new_scaled = scaler.transform(X_new)
        predictions = model.predict(X_new_scaled)
        probabilities = model.predict_proba(X_new_scaled)[:, 1]
    else:
        predictions = model.predict(X_new)
        probabilities = model.predict_proba(X_new)[:, 1]
    
    return predictions, probabilities, feature_status

def _impute_missing_features_saved_model(new_data, missing_features, label_encoders, feature_medians, all_features):
    """Impute missing features for saved model predictions"""
    new_data_copy = new_data.copy()
    
    print(f"Imputing {len(missing_features)} missing features...")
    
    for feature in missing_features:
        if feature in feature_medians:
            # Use stored median/mode from training
            imputed_value = feature_medians[feature]
            print(f"  {feature}: imputed with training value {imputed_value}")
        else:
            # Use neutral value for unknown features
            imputed_value = 0
            print(f"  {feature}: unknown feature, imputing with 0")
        
        new_data_copy[feature] = imputed_value
    
    return new_data_copy



In [ ]:
# Example usage using real data
def main():
    """
    Complete example showing training and prediction workflow
    """
    
    # Create sample training data
    np.random.seed(42)
    
    
    df_train = df_20_raw_per.copy()
    
    print("="*60)
    print("COMPLETE WORK-FROM-HOME PREDICTION WORKFLOW")
    print("="*60)
    
    # STEP 1: Feature Selection and Model Training
    print("\n1. TRAINING PHASE")
    print("-" * 30)
    
    selector = WorkFromHomeFeatureSelector(df_train, 'WfhCor')
    results = selector.run_complete_analysis()
    
    # Train final model
    model, features = selector.train_final_model(method='consensus', model_type='random_forest')
    
    # Save model for future use
    selector.save_model_pipeline('wfh_model.pkl')
    
     # STEP 2: Create new unseen data for prediction
    print("\n2. PREDICTION ON NEW DATA")
    print("-" * 30)
    
    # Simulate new data (same structure but without target variable)
    np.random.seed(123)  # Different seed for new data
    
    
    df_new = df_19_raw_per.copy()
    
    # Method 1: Using the trained selector object
    predictions, probabilities, features = selector.predict_new_data(df_new)
    
    # Create results dataframe
    results_df = df_new.copy()
    results_df['predicted_wfh'] = predictions
    results_df['wfh_probability'] = probabilities
    results_df['wfh_prediction'] = results_df['predicted_wfh'].map({0: 'Cannot WFH', 1: 'Can WFH'})
    
    print(f"Predictions completed for {len(df_new)} samples")
    print(f"Predicted WFH capability: {predictions.sum()} out of {len(predictions)} ({predictions.mean():.1%})")
    
    # Show sample predictions
    print('\nIf stopped here, here is the error')
    print("\nSample Predictions:")
    print(results_df[[ 'wfh_probability']].head(10))
    print(results_df[[ 'wfh_prediction']].head(10))
#    print(results_df[['age', 'education_level', 'occupation_category', 'wfh_prediction', 'wfh_probability']].head(10))
    
    # STEP 3: Using saved model (for production use)
    print("\n3. USING SAVED MODEL")
    print("-" * 30)
    
    # Load saved model
    saved_pipeline = load_saved_model('wfh_model.pkl')
    
    # Make predictions with saved model
    saved_predictions, saved_probabilities, saved_features = predict_with_saved_model(saved_pipeline, df_new)
    
    print(f"Saved model predictions match: {np.array_equal(predictions, saved_predictions)}")
    
    # Return the selector and results for further use
    return selector, results_df


def missing_features_example():
    """
    Demonstrate handling of missing features in new data
    """
    print("\n" + "="*60)
    print("MISSING FEATURES HANDLING EXAMPLE")
    print("="*60)
    
    # Create training data with more features
    np.random.seed(42)
    
    
    df_train = df_20_raw_per.copy()
    
    # Train model
    print("1. Training model with full feature set...")
    selector = WorkFromHomeFeatureSelector(df_train, 'WfhCor')
    selector.run_complete_analysis()
    model, features = selector.train_final_model(method='consensus')
    
    print(f"Selected features: {features}")
    
    # Create new data with missing features
    np.random.seed(123)
    n_new = 50
    
    # Scenario 1: Missing some features
    incomplete_data = df_19_raw_per.copy()
    
    print(f"\n2. Testing with incomplete data ({incomplete_data.shape[1]} out of {len(features)} features)...")
    
    # Method 1: Impute missing features (recommended)
    print("\n--- Method 1: Impute Missing Features ---")
    try:
        predictions1, probabilities1, status1 = selector.predict_new_data(
            incomplete_data, handle_missing_features='impute'
        )
        print(f"Success! Predictions made for {len(predictions1)} samples")
        print(f"WFH Rate: {predictions1.mean():.1%}")
        print(f"Average confidence: {probabilities1.mean():.1%}")
        
        # Show which features were imputed
        if status1['missing_features']:
            print("Imputed features and their importance:")
            importance_info = selector.get_feature_importance_for_missing(status1['missing_features'])
            for feature in status1['missing_features']:
                if importance_info and feature in importance_info:
                    print(f"  {feature}: RF importance = {importance_info[feature]['rf_importance']:.3f}")
    
    except Exception as e:
        print(f"Error with imputation: {e}")
    
    # Method 2: Retrain with available features
    print("\n--- Method 2: Retrain with Available Features ---")
    try:
        predictions2, probabilities2, status2 = selector.predict_new_data(
            incomplete_data, handle_missing_features='retrain'
        )
        print(f"Success! Predictions made for {len(predictions2)} samples")
        print(f"WFH Rate: {predictions2.mean():.1%}")
        print(f"Average confidence: {probabilities2.mean():.1%}")
        if 'warning' in status2:
            print(f"Warning: {status2['warning']}")
    
    except Exception as e:
        print(f"Error with retraining: {e}")
    
    # Method 3: Error on missing features
    print("\n--- Method 3: Strict Mode (Error on Missing) ---")
    try:
        predictions3, probabilities3, status3 = selector.predict_new_data(
            incomplete_data, handle_missing_features='error'
        )
    except Exception as e:
        print(f"Expected error: {e}")
    
    # Scenario 2: Extremely limited data
    print(f"\n3. Testing with very limited data...")
    
    very_limited_data = pd.DataFrame({
        'age': np.random.randint(25, 45, n_new),
        'education_level': np.random.choice(['Bachelor', 'Master'], n_new),
        # Only 2 features available
    })
    
    print(f"Very limited data: {very_limited_data.shape[1]} features")
    
    try:
        predictions_limited, _, status_limited = selector.predict_new_data(
            very_limited_data, handle_missing_features='impute'
        )
        print(f"Success with heavy imputation! Coverage: {status_limited['coverage']:.1%}")
        print(f"WFH Rate: {predictions_limited.mean():.1%}")
    except Exception as e:
        print(f"Error with very limited data: {e}")
    
    # Scenario 3: Using saved model with missing features
    print(f"\n4. Testing saved model with missing features...")
    
    # Save and reload model
    selector.save_model_pipeline('wfh_model_missing_test.pkl')
    pipeline = load_saved_model('wfh_model_missing_test.pkl')
    
    try:
        predictions_saved, probabilities_saved, status_saved = predict_with_saved_model(
            pipeline, incomplete_data, handle_missing_features='impute'
        )
        print(f"Saved model success! Coverage: {status_saved['coverage']:.1%}")
        print(f"Results match: {np.array_equal(predictions1, predictions_saved)}")
    except Exception as e:
        print(f"Error with saved model: {e}")
    
    return incomplete_data, predictions1, probabilities1, status1

if __name__ == "__main__":
    # Run complete example
    selector, results = main()
    
    # Demonstrate missing features handling
    missing_features_example()

In [31]:
# Demonstrate handling of missing features in new data
# Train the model

print("\n" + "="*60)
print("MISSING FEATURES HANDLING EXAMPLE")
print("="*60)

# Create training data with more features
np.random.seed(42)


df_train = df_20_raw_per.copy()

# Train model
print("1. Training model with full feature set...")
selector = WorkFromHomeFeatureSelector(df_train, 'WfhCor')
selector.run_complete_analysis()
model, features = selector.train_final_model(method='consensus')

print(f"Selected features: {features}")

# Create new data with missing features
np.random.seed(123)
n_new = 50

    


MISSING FEATURES HANDLING EXAMPLE
1. Training model with full feature set...
WORK FROM HOME FEATURE SELECTION ANALYSIS
Preprocessing data...
Data shape: (12237, 943)
Target distribution:
WfhCor
0    9837
1    2400
Name: count, dtype: int64

=== Mutual Information Feature Selection (top 20) ===
Top 10 features by Mutual Information:
         feature  mi_score
888       DVPayp  0.220942
187       DVnino  0.215836
907  dvgrosspayp  0.213103
900      DVTAX1p  0.212746
871      SICCODE  0.162705
876       EmpStY  0.158682
78        Wrking  0.144918
915      GWkIncp  0.122190
916      dvage_p  0.119450
99        DVUsHr  0.118391

=== Random Forest Feature Selection (top 20) ===
Top 10 features by Random Forest importance:
         feature  importance
900      DVTAX1p    0.043345
915      GWkIncp    0.038873
907  dvgrosspayp    0.036841
888       DVPayp    0.036767
187       DVnino    0.036607
78        Wrking    0.025817
871      SICCODE    0.025578
906    dedpenamp    0.023232
197      PyD

In [38]:
# Predicting the WFH indicator of 2019 data

# Scenario 1: Missing some features
incomplete_data = df_19_raw_per.copy()

print(f"\n2. Testing with incomplete data ({incomplete_data.shape[1]} out of {len(features)} features)...")

# Method 1: Impute missing features (recommended)
print("\n--- Method 1: Impute Missing Features ---")
try:
    predictions1, probabilities1, status1 = selector.predict_new_data(
        incomplete_data, handle_missing_features='impute'
    )
    
    # Create results dataframe
    results_df = incomplete_data.copy()
    results_df['predicted_wfh'] = predictions1
    results_df['wfh_probability'] = probabilities1
    #results_df['wfh_prediction'] = results_df['predicted_wfh'].map({0: 'Cannot WFH', 1: 'Can WFH'})
    
    print(f"Success! Predictions made for {len(predictions1)} samples")
    print(f"WFH Rate: {predictions1.mean():.1%}")
    print(f"Average confidence: {probabilities1.mean():.1%}")
    
    # Show predictions
    print('\nIf stopped here, here is the error')
    print("\nSample Predictions:")
    print(results_df[[ 'wfh_probability']].head(10))
    print(results_df[[ 'predicted_wfh']].head(10))
    
    # Show which features were imputed
    if status1['missing_features']:
        print("Imputed features and their importance:")
        importance_info = selector.get_feature_importance_for_missing(status1['missing_features'])
        for feature in status1['missing_features']:
            if importance_info and feature in importance_info:
                print(f"  {feature}: RF importance = {importance_info[feature]['rf_importance']:.3f}")

except Exception as e:
    print(f"Error with imputation: {e}")


2. Testing with incomplete data (945 out of 19 features)...

--- Method 1: Impute Missing Features ---
Making predictions for 12658 samples...
Feature coverage: 94.7% (18/19)
Missing features: ['ClaWfhCor_p']
Imputing 1 missing features...
  ClaWfhCor_p: imputed with median value 2.0
Success! Predictions made for 12658 samples
WFH Rate: 22.9%
Average confidence: 44.9%

If stopped here, here is the error

Sample Predictions:
   wfh_probability
0         0.488865
1         0.330214
2         0.394589
3         0.183176
4         0.601255
5         0.336407
6         0.557354
7         0.574756
8         0.373730
9         0.401591
   predicted_wfh
0              0
1              0
2              0
3              0
4              1
5              0
6              1
7              1
8              0
9              0
Imputed features and their importance:
  ClaWfhCor_p: RF importance = 0.016


In [39]:
# Save pseudo 2019 data
df_19_raw_per_pseudo = results_df.copy()
pyrs.write_sav(df_19_raw_per_pseudo, 'lcfs_2019_rawper_pseudo.sav')

In [40]:
# Predicting the WFH indicator of 2018 data

# Scenario 1: Missing some features
incomplete_data = df_18_raw_per.copy()

print(f"\n2. Testing with incomplete data ({incomplete_data.shape[1]} out of {len(features)} features)...")

# Method 1: Impute missing features (recommended)
print("\n--- Method 1: Impute Missing Features ---")
try:
    predictions1, probabilities1, status1 = selector.predict_new_data(
        incomplete_data, handle_missing_features='impute'
    )
    
    # Create results dataframe
    results_df = incomplete_data.copy()
    results_df['predicted_wfh'] = predictions1
    results_df['wfh_probability'] = probabilities1
    #results_df['wfh_prediction'] = results_df['predicted_wfh'].map({0: 'Cannot WFH', 1: 'Can WFH'})
    
    print(f"Success! Predictions made for {len(predictions1)} samples")
    print(f"WFH Rate: {predictions1.mean():.1%}")
    print(f"Average confidence: {probabilities1.mean():.1%}")
    
    # Show predictions
    print('\nIf stopped here, here is the error')
    print("\nSample Predictions:")
    print(results_df[[ 'wfh_probability']].head(10))
    print(results_df[[ 'predicted_wfh']].head(10))
    
    # Show which features were imputed
    if status1['missing_features']:
        print("Imputed features and their importance:")
        importance_info = selector.get_feature_importance_for_missing(status1['missing_features'])
        for feature in status1['missing_features']:
            if importance_info and feature in importance_info:
                print(f"  {feature}: RF importance = {importance_info[feature]['rf_importance']:.3f}")

except Exception as e:
    print(f"Error with imputation: {e}")


2. Testing with incomplete data (970 out of 19 features)...

--- Method 1: Impute Missing Features ---
Making predictions for 12763 samples...
Feature coverage: 94.7% (18/19)
Missing features: ['ClaWfhCor_p']
Imputing 1 missing features...
  ClaWfhCor_p: imputed with median value 2.0
Success! Predictions made for 12763 samples
WFH Rate: 19.4%
Average confidence: 42.7%

If stopped here, here is the error

Sample Predictions:
   wfh_probability
0         0.635521
1         0.573231
2         0.581858
3         0.619152
4         0.437720
5         0.437720
6         0.251349
7         0.177090
8         0.398594
9         0.346051
   predicted_wfh
0              1
1              1
2              1
3              1
4              0
5              0
6              0
7              0
8              0
9              0
Imputed features and their importance:
  ClaWfhCor_p: RF importance = 0.016


In [41]:
# Save pseudo 2018 data
df_18_raw_per_pseudo = results_df.copy()
pyrs.write_sav(df_18_raw_per_pseudo, 'lcfs_2018_rawper_pseudo.sav')

In [42]:
# Predicting the WFH indicator of 2017 data

# Scenario 1: Missing some features
incomplete_data = df_17_raw_per.copy()

print(f"\n2. Testing with incomplete data ({incomplete_data.shape[1]} out of {len(features)} features)...")

# Method 1: Impute missing features (recommended)
print("\n--- Method 1: Impute Missing Features ---")
try:
    predictions1, probabilities1, status1 = selector.predict_new_data(
        incomplete_data, handle_missing_features='impute'
    )
    
    # Create results dataframe
    results_df = incomplete_data.copy()
    results_df['predicted_wfh'] = predictions1
    results_df['wfh_probability'] = probabilities1
    #results_df['wfh_prediction'] = results_df['predicted_wfh'].map({0: 'Cannot WFH', 1: 'Can WFH'})
    
    print(f"Success! Predictions made for {len(predictions1)} samples")
    print(f"WFH Rate: {predictions1.mean():.1%}")
    print(f"Average confidence: {probabilities1.mean():.1%}")
    
    # Show predictions
    print('\nIf stopped here, here is the error')
    print("\nSample Predictions:")
    print(results_df[[ 'wfh_probability']].head(10))
    print(results_df[[ 'predicted_wfh']].head(10))
    
    # Show which features were imputed
    if status1['missing_features']:
        print("Imputed features and their importance:")
        importance_info = selector.get_feature_importance_for_missing(status1['missing_features'])
        for feature in status1['missing_features']:
            if importance_info and feature in importance_info:
                print(f"  {feature}: RF importance = {importance_info[feature]['rf_importance']:.3f}")

except Exception as e:
    print(f"Error with imputation: {e}")


2. Testing with incomplete data (894 out of 19 features)...

--- Method 1: Impute Missing Features ---
Making predictions for 12753 samples...
Feature coverage: 78.9% (15/19)
Missing features: ['dvgrosspayp', 'dedpenamp', 'ClaWfhCor_p', 'EdAgeCor']
Imputing 4 missing features...
  dvgrosspayp: imputed with median value 473.11
  dedpenamp: imputed with median value 100.0
  ClaWfhCor_p: imputed with median value 2.0
  EdAgeCor: imputed with median value 18.0
Success! Predictions made for 12753 samples
WFH Rate: 20.8%
Average confidence: 41.3%

If stopped here, here is the error

Sample Predictions:
   wfh_probability
0         0.386917
1         0.337589
2         0.411641
3         0.473323
4         0.467454
5         0.411641
6         0.411641
7         0.310489
8         0.312494
9         0.306239
   predicted_wfh
0              0
1              0
2              0
3              0
4              0
5              0
6              0
7              0
8              0
9              0

In [43]:
# Save pseudo 2017 data
df_17_raw_per_pseudo = results_df.copy()
pyrs.write_sav(df_17_raw_per_pseudo, 'lcfs_2017_rawper_pseudo.sav')

In [44]:
# Predicting the WFH indicator of 2016 data

# Scenario 1: Missing some features
incomplete_data = df_16_raw_per.copy()

print(f"\n2. Testing with incomplete data ({incomplete_data.shape[1]} out of {len(features)} features)...")

# Method 1: Impute missing features (recommended)
print("\n--- Method 1: Impute Missing Features ---")
try:
    predictions1, probabilities1, status1 = selector.predict_new_data(
        incomplete_data, handle_missing_features='impute'
    )
    
    # Create results dataframe
    results_df = incomplete_data.copy()
    results_df['predicted_wfh'] = predictions1
    results_df['wfh_probability'] = probabilities1
    #results_df['wfh_prediction'] = results_df['predicted_wfh'].map({0: 'Cannot WFH', 1: 'Can WFH'})
    
    print(f"Success! Predictions made for {len(predictions1)} samples")
    print(f"WFH Rate: {predictions1.mean():.1%}")
    print(f"Average confidence: {probabilities1.mean():.1%}")
    
    # Show predictions
    print('\nIf stopped here, here is the error')
    print("\nSample Predictions:")
    print(results_df[[ 'wfh_probability']].head(10))
    print(results_df[[ 'predicted_wfh']].head(10))
    
    # Show which features were imputed
    if status1['missing_features']:
        print("Imputed features and their importance:")
        importance_info = selector.get_feature_importance_for_missing(status1['missing_features'])
        for feature in status1['missing_features']:
            if importance_info and feature in importance_info:
                print(f"  {feature}: RF importance = {importance_info[feature]['rf_importance']:.3f}")

except Exception as e:
    print(f"Error with imputation: {e}")


2. Testing with incomplete data (1301 out of 19 features)...

--- Method 1: Impute Missing Features ---
Making predictions for 11938 samples...
Feature coverage: 73.7% (14/19)
Missing features: ['ClaWfhCor_p', 'GWkIncp', 'dedpenamp', 'dvgrosspayp', 'EdAgeCor']
Imputing 5 missing features...
  ClaWfhCor_p: imputed with median value 2.0
  GWkIncp: imputed with median value 394.33000000000004
  dedpenamp: imputed with median value 100.0
  dvgrosspayp: imputed with median value 473.11
  EdAgeCor: imputed with median value 18.0
Success! Predictions made for 11938 samples
WFH Rate: 15.3%
Average confidence: 38.3%

If stopped here, here is the error

Sample Predictions:
   wfh_probability
0         0.489804
1         0.263577
2         0.520864
3         0.153445
4         0.531869
5         0.395250
6         0.273157
7         0.311068
8         0.557119
9         0.259474
   predicted_wfh
0              0
1              0
2              1
3              0
4              1
5              0

In [45]:
# Save pseudo 2016 data
df_16_raw_per_pseudo = results_df.copy()
pyrs.write_sav(df_16_raw_per_pseudo, 'lcfs_2016_rawper_pseudo.sav')